In [ ]:
import pandas as pd
import numpy as np 
import pandas as pd
from fbprophet import Prophet

In [ ]:
df = pd.read_csv("faturamento_lojas_atuais.csv")

df['dateInt'] = df['ano'].astype(str) + '-' + df['mes'].astype(str)
df['data'] = pd.to_datetime(df['dateInt'])
df.drop(['dateInt','ano','mes'],axis=1,inplace=True)
df = df[['data','cod_loja','categoria','qtde','receita']]
df.index = df['data']
df.drop('data',axis=1,inplace=True)

In [ ]:
df_n = df.groupby('cod_loja').resample('M')['receita'].agg('mean')
df_n = df_n.reset_index()
df_n = df_n.set_index('data')

In [ ]:
df_n["receita"] = df_n.groupby("cod_loja").transform(lambda x: x.fillna(x.mean()))

In [ ]:
df_simples = df_n.query('cod_loja >= 3274').groupby('cod_loja').agg({'receita':'mean'})

In [ ]:
df_n.head()

,cod_loja,receita
data,,
2016-01-31,1,31559.526429
2016-02-29,1,28238.587143
2016-03-31,1,41986.441429
2016-04-30,1,38187.060000
2016-05-31,1,42534.137143


In [ ]:
datas = ['2018-01', '2018-02', '2018-03']
datas = pd.to_datetime(datas, format='%Y-%m')
datas

DatetimeIndex(['2018-01-01', '2018-02-01', '2018-03-01'], dtype='datetime64[ns]', freq=None)

In [ ]:
serie = df_n['cod_loja'].unique()
ausentes = []
for i in range(1,3688):
    if i not in serie:
        ausentes.append(i) 

In [ ]:
y_media = []
for i in range(3275, 3688):
  if i not in ausentes:
    df = pd.DataFrame({'ds': datas, 'cod_loja': i, 'receita': df_simples['receita'][i]})
  y_media.append(df)
print(y_media)

In [ ]:
y_medio = pd.concat(y_media)
y_medio
y_medio.to_csv('y_medio_3275prafrente.csv')

In [ ]:
y_medio_ = pd.read_csv('y_medio_3275prafrente.csv')
y_previsto = pd.read_csv('previsao_ate_3273.csv')

In [ ]:
y_medio_.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
y_completo = pd.merge(y_previsto, y_medio_, how='outer', left_on=['ds', 'cod_loja', 'receita'], right_on=['ds', 'cod_loja', 'receita'])

In [ ]:
y_completo.to_csv('y_completo.csv')

In [ ]:
y_completo

,ds,cod_loja,receita
0,2018-01-01,1,80951.299259
1,2018-02-01,1,63730.368546
2,2018-03-01,1,62222.296678
3,2018-01-01,2,6452.550443
4,2018-02-01,2,5543.748073
...,...,...,...
9592,2018-02-01,3686,7298.622912
9593,2018-03-01,3686,7298.622912
9594,2018-01-01,3687,4868.569208
9595,2018-02-01,3687,4868.569208


# Modelo


In [ ]:
!easy_install fancyimpute 

In [ ]:
lojas_atuais = pd.read_csv('lojas_atuais_completo.csv')
cenarios = pd.read_csv('categoricas_expansão.csv')

In [ ]:
from fancyimpute import KNN
KNN_imputer = KNN()

num_features = ['cod_municipio', 'feature_16', 'feature_17', 'feature_13', 'feature_14', 'feature_15', 'feature_18','feature_04', 'feature_06', 'feature_07', 'feature_09', 'feature_10' ]
df_knn = cenarios.copy()
df_knn = KNN_imputer.fit_transform(df_knn[num_features])
df_knn

Imputing row 1/553 with 0 missing, elapsed time: 0.056
Imputing row 101/553 with 6 missing, elapsed time: 0.060
Imputing row 201/553 with 6 missing, elapsed time: 0.067
Imputing row 301/553 with 6 missing, elapsed time: 0.074
Imputing row 401/553 with 6 missing, elapsed time: 0.081
Imputing row 501/553 with 6 missing, elapsed time: 0.089


array([[2.93330700e+06, 1.00000000e+02, 7.17000000e+01, ...,
        1.34654243e-01, 1.16194058e+02, 2.91349824e+00],
       [2.91640100e+06, 1.00000000e+02, 6.35000000e+01, ...,
        1.16662026e-01, 2.63457420e+01, 1.31261157e+00],
       [2.92870300e+06, 6.84802060e+01, 7.57947946e+01, ...,
        1.04162104e-01, 3.53627882e+01, 1.96951195e+00],
       ...,
       [5.20870700e+06, 1.00000000e+02, 6.06000000e+01, ...,
        1.90896443e-01, 6.61415873e+02, 2.79584007e+00],
       [4.21820200e+06, 1.00000000e+02, 9.42000000e+01, ...,
        1.33753965e-01, 2.00279086e+01, 2.42230458e+00],
       [2.91760700e+06, 5.00000000e+01, 9.97000000e+01, ...,
        1.23647602e-01, 1.47815098e+01, 1.80345903e+00]])

In [ ]:
cenarios.drop(columns=num_features, inplace=True)

In [ ]:
cenarios

,Unnamed: 0,cod_loja,cod_ap,feature_01,feature_02,feature_03,feature_05,feature_08,feature_11,feature_12
0,0,10,2933307005008,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,False,FEATURE_05_VALUE_03,0.131481,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01
1,1,13,2916401003002,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,FEATURE_05_VALUE_03,0.114995,FEATURE_11_VALUE_04,FEATURE_12_VALUE_01
2,2,16,2928703003003,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,FEATURE_05_VALUE_05,0.087807,FEATURE_11_VALUE_03,FEATURE_12_VALUE_01
3,3,21,2906006003001,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,FEATURE_05_VALUE_03,0.131330,FEATURE_11_VALUE_06,FEATURE_12_VALUE_02
4,4,28,3550308005058,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,True,FEATURE_05_VALUE_03,0.179140,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01
...,...,...,...,...,...,...,...,...,...,...
548,548,3667,2927408005009,FEATURE_01_VALUE_03,FEATURE_02_VALUE_03,True,FEATURE_05_VALUE_06,0.096627,FEATURE_11_VALUE_02,FEATURE_12_VALUE_02
549,549,3672,-1,FEATURE_01_VALUE_08,FEATURE_02_VALUE_03,False,FEATURE_05_VALUE_05,0.179423,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01
550,550,3674,5208707005001,FEATURE_01_VALUE_08,FEATURE_02_VALUE_03,False,FEATURE_05_VALUE_05,0.179423,FEATURE_11_VALUE_03,FEATURE_12_VALUE_01
551,551,3675,4218202003002,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,FEATURE_05_VALUE_03,0.134264,FEATURE_11_VALUE_05,FEATURE_12_VALUE_02


In [ ]:
numeric_expansao = pd.DataFrame(df_knn)
numeric_expansao.columns = num_features
cenarios_expansao = pd.merge(numeric_expansao, cenarios, left_index=True, right_index=True)
cenarios_expansao.drop(columns=['Unnamed: 0', 'cod_ap'], inplace=True)
'''
cenarios_expansao = cenarios_expansao[['cod_municipio', 'cod_loja', 'feature_01', 'feature_02', 'feature_03',
                      'feature_05', 'feature_06', 'feature_07', 'feature_08', 'feature_09', 
                      'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
                      'feature_15', 'feature_16', 'feature_17', 'feature_18']]#
'''
df_predict=pd.get_dummies(cenarios_expansao, columns=['feature_01', 'feature_02', 'feature_03', 'feature_05', 'feature_11', 'feature_12'], drop_first=True)

In [ ]:
csv_expansao = cenarios_expansao.copy()
csv_expansao.drop(columns='receita_m_prevista', inplace=True)
csv_expansao.to_csv('csv_expansao.csv')

In [ ]:
df_receitas_m = y_completo.groupby('cod_loja')['receita'].agg('mean')

In [ ]:
df_receitas_m = df_receitas_m.reset_index()

In [ ]:
df_atual_prev = pd.merge(df_receitas_m, lojas_atuais, left_index=True, right_index=True)

In [ ]:
df_atual_prev.drop(columns=['Unnamed: 0','Column2', 'cod_ap'], inplace=True)

In [ ]:
df_atual_prev

,cod_loja,receita,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18
0,1,68967.988161,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.168922,0.167659,115.958848,2.349821,FEATURE_11_VALUE_03,FEATURE_12_VALUE_01,99.863577,81.684705,99.999997,99.999997,84.099721,91.187920
1,2,5629.142177,5300108,FEATURE_01_VALUE_10,FEATURE_02_VALUE_03,True,22.00,FEATURE_05_VALUE_03,27,0.215967,0.197287,1562.674817,3.670883,FEATURE_11_VALUE_04,FEATURE_12_VALUE_01,95.005777,88.874647,60.873485,86.965818,67.656491,80.161129
2,3,20015.035860,3550308,FEATURE_01_VALUE_10,FEATURE_02_VALUE_03,True,45.66,FEATURE_05_VALUE_03,6,0.190102,0.179140,5593.905463,2.815883,FEATURE_11_VALUE_03,FEATURE_12_VALUE_02,94.287134,91.097912,92.021671,99.999996,93.022955,94.325879
3,4,7400.897471,3145604,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,30.00,FEATURE_05_VALUE_03,3,0.141988,0.143677,12.824022,7.218132,FEATURE_11_VALUE_03,FEATURE_12_VALUE_02,98.596858,84.494294,98.601741,100.000000,91.884291,94.218696
4,5,20276.497948,1600303,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,False,62.49,FEATURE_05_VALUE_03,30,0.107619,0.103083,325.531120,1.972944,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.000000,88.100000,100.000000,100.000000,88.500000,93.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3125,3682,11243.960357,5105259,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,30.16,FEATURE_05_VALUE_03,5,0.210289,0.167499,36.397933,3.491315,FEATURE_11_VALUE_06,FEATURE_12_VALUE_02,100.000000,91.700000,100.000000,100.000000,97.100000,97.600000
3126,3683,12601.095124,2604106,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,False,38.00,FEATURE_05_VALUE_03,13,0.146264,0.152495,163.567629,2.016780,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.000000,85.000000,100.000000,100.000000,50.700000,72.500000
3127,3685,15115.522717,4115804,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,44.04,FEATURE_05_VALUE_02,7,0.226926,0.240253,16.862438,4.455832,FEATURE_11_VALUE_04,FEATURE_12_VALUE_01,100.000000,71.500000,100.000000,100.000000,99.000000,95.200000
3128,3686,7298.622912,3509700,FEATURE_01_VALUE_10,FEATURE_02_VALUE_06,False,31.76,FEATURE_05_VALUE_03,4,0.134354,0.136028,20.565405,3.932982,FEATURE_11_VALUE_03,FEATURE_12_VALUE_02,88.463863,81.120704,74.681917,75.315017,67.765410,77.445629


In [ ]:
df_model = pd.get_dummies(df_atual_prev, columns=['feature_01', 'feature_02', 'feature_03', 'feature_05', 'feature_11', 'feature_12'], drop_first=True)
df_model.rename(columns={'receita': 'receita_m_prevista'}, inplace=True)
df_model.drop(columns='')

,cod_loja,receita_m_prevista,cod_municipio,feature_04,feature_06,feature_07,feature_08,feature_09,feature_10,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_01_FEATURE_01_VALUE_06,feature_01_FEATURE_01_VALUE_08,feature_01_FEATURE_01_VALUE_10,feature_02_FEATURE_02_VALUE_03,feature_02_FEATURE_02_VALUE_04,feature_02_FEATURE_02_VALUE_06,feature_03_True,feature_05_FEATURE_05_VALUE_02,feature_05_FEATURE_05_VALUE_03,feature_05_FEATURE_05_VALUE_04,feature_05_FEATURE_05_VALUE_05,feature_05_FEATURE_05_VALUE_06,feature_05_FEATURE_05_VALUE_07,feature_05_FEATURE_05_VALUE_08,feature_11_FEATURE_11_VALUE_02,feature_11_FEATURE_11_VALUE_03,feature_11_FEATURE_11_VALUE_04,feature_11_FEATURE_11_VALUE_05,feature_11_FEATURE_11_VALUE_06,feature_11_FEATURE_11_VALUE_07,feature_12_FEATURE_12_VALUE_02
0,1,68967.988161,4125506,21.76,31,0.168922,0.167659,115.958848,2.349821,99.863577,81.684705,99.999997,99.999997,84.099721,91.187920,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1,2,5629.142177,5300108,22.00,27,0.215967,0.197287,1562.674817,3.670883,95.005777,88.874647,60.873485,86.965818,67.656491,80.161129,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,3,20015.035860,3550308,45.66,6,0.190102,0.179140,5593.905463,2.815883,94.287134,91.097912,92.021671,99.999996,93.022955,94.325879,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1
3,4,7400.897471,3145604,30.00,3,0.141988,0.143677,12.824022,7.218132,98.596858,84.494294,98.601741,100.000000,91.884291,94.218696,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
4,5,20276.497948,1600303,62.49,30,0.107619,0.103083,325.531120,1.972944,100.000000,88.100000,100.000000,100.000000,88.500000,93.800000,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3125,3682,11243.960357,5105259,30.16,5,0.210289,0.167499,36.397933,3.491315,100.000000,91.700000,100.000000,100.000000,97.100000,97.600000,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1
3126,3683,12601.095124,2604106,38.00,13,0.146264,0.152495,163.567629,2.016780,100.000000,85.000000,100.000000,100.000000,50.700000,72.500000,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
3127,3685,15115.522717,4115804,44.04,7,0.226926,0.240253,16.862438,4.455832,100.000000,71.500000,100.000000,100.000000,99.000000,95.200000,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3128,3686,7298.622912,3509700,31.76,4,0.134354,0.136028,20.565405,3.932982,88.463863,81.120704,74.681917,75.315017,67.765410,77.445629,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
X_train, X_test, y_train, y_test = train_test_split(df_model.drop('receita_m_prevista', axis=1),
                                                    df_model['receita_m_prevista'],
                                                    random_state=42, test_size=0.25)

In [ ]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
df_predict['feature_dummie'] = 0

In [ ]:
df_model.columns

Index(['cod_loja', 'receita_m_prevista', 'cod_municipio', 'feature_04',
       'feature_06', 'feature_07', 'feature_08', 'feature_09', 'feature_10',
       'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17',
       'feature_18', 'feature_01_FEATURE_01_VALUE_06',
       'feature_01_FEATURE_01_VALUE_08', 'feature_01_FEATURE_01_VALUE_10',
       'feature_02_FEATURE_02_VALUE_03', 'feature_02_FEATURE_02_VALUE_04',
       'feature_02_FEATURE_02_VALUE_06', 'feature_03_True',
       'feature_05_FEATURE_05_VALUE_02', 'feature_05_FEATURE_05_VALUE_03',
       'feature_05_FEATURE_05_VALUE_04', 'feature_05_FEATURE_05_VALUE_05',
       'feature_05_FEATURE_05_VALUE_06', 'feature_05_FEATURE_05_VALUE_07',
       'feature_05_FEATURE_05_VALUE_08', 'feature_11_FEATURE_11_VALUE_02',
       'feature_11_FEATURE_11_VALUE_03', 'feature_11_FEATURE_11_VALUE_04',
       'feature_11_FEATURE_11_VALUE_05', 'feature_11_FEATURE_11_VALUE_06',
       'feature_11_FEATURE_11_VALUE_07', 'feature_12_FEATUR

In [ ]:
cenarios_expansao['receita_m_prevista'] = model.predict(df_predict)

In [ ]:
cenarios_expansao

,cod_municipio,feature_16,feature_17,feature_13,feature_14,feature_15,feature_18,feature_04,feature_06,feature_07,feature_09,feature_10,cod_loja,feature_01,feature_02,feature_03,feature_05,feature_08,feature_11,feature_12,receita_m_prevista
0,2933307.0,100.000000,71.700000,100.000000,93.400000,100.000000,88.800000,50.00,6.0,0.134654,116.194058,2.913498,10,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,False,FEATURE_05_VALUE_03,0.131481,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,23116.276344
1,2916401.0,100.000000,63.500000,84.300000,77.500000,100.000000,78.800000,41.00,4.0,0.116662,26.345742,1.312612,13,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,FEATURE_05_VALUE_03,0.114995,FEATURE_11_VALUE_04,FEATURE_12_VALUE_01,24213.406404
2,2928703.0,68.480206,75.794795,86.969705,81.829849,97.174439,79.219759,42.12,2.0,0.104162,35.362788,1.969512,16,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,FEATURE_05_VALUE_05,0.087807,FEATURE_11_VALUE_03,FEATURE_12_VALUE_01,24282.119222
3,2906006.0,100.000000,100.000000,85.000000,76.900000,100.000000,92.900000,30.00,6.0,0.132954,9.862757,1.383681,21,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,FEATURE_05_VALUE_03,0.131330,FEATURE_11_VALUE_06,FEATURE_12_VALUE_02,34404.705446
4,3550308.0,99.999994,94.644681,96.498460,69.129007,89.729653,91.793706,44.31,8.0,0.190102,5593.905463,2.815883,28,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,True,FEATURE_05_VALUE_03,0.179140,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,25137.525735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,2927408.0,84.254933,73.494461,92.218531,85.241140,78.058997,80.789746,11.47,3.0,0.107963,1355.486603,4.142294,3667,FEATURE_01_VALUE_03,FEATURE_02_VALUE_03,True,FEATURE_05_VALUE_06,0.096627,FEATURE_11_VALUE_02,FEATURE_12_VALUE_02,23200.588739
549,-1.0,99.999999,95.349888,99.517782,71.829595,67.984217,92.627816,31.00,5.0,0.190896,32.409193,2.989030,3672,FEATURE_01_VALUE_08,FEATURE_02_VALUE_03,False,FEATURE_05_VALUE_05,0.179423,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,33703.778388
550,5208707.0,100.000000,60.600000,100.000000,78.300000,100.000000,81.100000,58.00,9.0,0.190896,661.415873,2.795840,3674,FEATURE_01_VALUE_08,FEATURE_02_VALUE_03,False,FEATURE_05_VALUE_05,0.179423,FEATURE_11_VALUE_03,FEATURE_12_VALUE_01,23954.620296
551,4218202.0,100.000000,94.200000,100.000000,92.500000,100.000000,96.600000,40.74,4.0,0.133754,20.027909,2.422305,3675,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,FEATURE_05_VALUE_03,0.134264,FEATURE_11_VALUE_05,FEATURE_12_VALUE_02,26569.524047


In [ ]:
predict_expansao = cenarios_expansao[['cod_loja', 'receita_m_prevista']]

In [ ]:
predict_expansao.to_csv('predict_expansao.csv')

In [ ]:
teste2 = predict_expansao.copy()
teste2.index = teste2['cod_loja']
teste2.drop(columns='cod_loja', inplace=True)
teste2.rename(columns={'receita_m_prevista':'faturamento'}, inplace=True)
#teste2 = teste2.reset_index()

In [ ]:
teste_1 = y_completo.groupby('cod_loja')['receita'].agg('mean')
teste_1 = teste_1.to_frame()
teste_1.rename(columns={'receita':'faturamento'}, inplace=True)
#teste_1 = teste_1.reset_index()

In [ ]:
frame = [teste_1, teste2]
d = pd.concat(frame)
d = d.reset_index()
desafio_2 = d.sort_values(['cod_loja']).head(15)
desafio_2 = desafio_2.reset_index()
desafio_2.drop(columns='index', inplace=True)
desafio_2.to_csv('desafio_2.csv')

In [ ]:
teste_1 = teste_1.reset_index()

In [ ]:
teste_1.to_csv('douglas.csv', sep=';')

In [ ]:
teste2 = teste2.reset_index()
teste2

,cod_loja,faturamento
0,10,23116.276344
1,13,24213.406404
2,16,24282.119222
3,21,34404.705446
4,28,25137.525735
...,...,...
548,3667,23200.588739
549,3672,33703.778388
550,3674,23954.620296
551,3675,26569.524047


In [ ]:
teste2.to_csv('desafio_2.csv', sep=';')

In [ ]:
print(f'Acc: {model.score(X_train, y_train)}')

Acc: 0.8643882818914067
